## Preprocessing

In [ ]:
! pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.2 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

from google.colab import drive
drive.mount('/content/drive')

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# check the keys or columns
application_df.keys()


Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df[[ 'APPLICATION_TYPE','NAME', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL']]

In [5]:
#check the keys
application_df.head()

,APPLICATION_TYPE,NAME,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,BLUE KNIGHTS MOTORCYCLE CLUB,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,ST CLOUD PROFESSIONAL FIREFIGHTERS,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,SOUTHSIDE ATHLETIC ASSOCIATION,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,GENETIC RESEARCH INSTITUTE OF THE DESERT,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE             17
NAME                      19568
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
at_counts = application_df['APPLICATION_TYPE'].value_counts()
at_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
name_counts = application_df['NAME'].value_counts()
name_counts

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [9]:
names_types_to_replace = name_counts[name_counts< 15].index
names_types_to_replace

Index(['NATIONAL KITCHEN & BATH ASSOC', 'MASTER GARDENER ASSOCIATION',
       'INTERNATIONAL BROTHERHOOD OF ELECTRICAL WORKERS',
       'COLLEGE AND UNIVERSITY PROFESSIONAL ASSOCIATION FOR HUMAN RESOURCES',
       'AMERICANS UNITED FOR SEPARATION OF CHURCH AND STATE',
       'GLOBAL SPORTS ALLIANCE INC', 'BEAVERHEAD COUNTY 4-H FOUNDATION',
       'ASSOCIATION OF CLINICAL RESEARCH PROFESSIONALS',
       'BOY SCOUTS OF AMERICA',
       'FLORIDA GOVERNMENT FINANCE OFFICERS ASSOCIATION INC',
       ...
       'PSSST', 'EASTERN OREGON WORKFORCE INVESTMENT BOARD',
       'OUR PERFECT STORM', 'RED SHOE PROJECT INC',
       'CHRISTIAN DEAF FELLOWSHIP', 'ST LOUIS SLAM WOMENS FOOTBALL',
       'AIESEC ALUMNI IBEROAMERICA CORP', 'WEALLBLEEDRED ORG INC',
       'AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI',
       'WATERHOUSE CHARITABLE TR'],
      dtype='object', length=19408)

In [10]:
for name in names_types_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                      21751
PARENT BOOSTER USA INC                      1260
TOPS CLUB INC                                765
UNITED STATES BOWLING CONGRESS INC           700
WASHINGTON STATE UNIVERSITY                  492
                                           ...  
CALIFORNIA FEDERATION OF WOMENS CLUBS         15
JAMESTOWNE SOCIETY                            15
THETA TAU                                     15
LAKE TRABIS ATHLETIC BOOSTER CLUB INC         15
INTERNATIONAL ASSOCIATION OF LIONS CLUB       15
Name: NAME, Length: 161, dtype: int64

In [11]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = at_counts[at_counts< 550].index
application_types_to_replace

Index(['T10', 'T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17'], dtype='object')

In [12]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [13]:
# Look at CLASSIFICATION value counts for binning
ct_counts = application_df['CLASSIFICATION'].value_counts()
ct_counts


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [14]:
# get counts greater than one
gt_counts = ct_counts[ct_counts>1]
gt_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [15]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ct_counts[ct_counts< 800].index
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [16]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [17]:
application_df.shape

(34299, 204)

In [18]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns = 'IS_SUCCESSFUL').values
y[:10]

0    1
1    1
2    0
3    1
4    1
5    1
6    1
7    1
8    1
9    0
Name: IS_SUCCESSFUL, dtype: int64

In [19]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,stratify = y)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
print(len(X_train_scaled))

25724


## Compile, Train and Evaluate the Model

In [21]:
input_features = len(X_train[0])
print(input_features)

203


In [22]:
from keras.layers import LeakyReLU
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units = 30,activation = 'relu', input_dim = input_features))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = 'relu'))
#nn.add(tf.keras.layers.Dense(units = 3, activation = 'relu'))
# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                6120      
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 6,441
Trainable params: 6,441
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
path = '/content/drive/MyDrive/weights.hdf5'
checkpoint = ModelCheckpoint(path, monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=5)

In [25]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs= 50, callbacks=[checkpoint])

Epoch 1/50
804/804 [==============================] - 7s 7ms/step - loss: 0.5038 - accuracy: 0.7563
Epoch 2/50
804/804 [==============================] - 5s 6ms/step - loss: 0.4536 - accuracy: 0.7778
Epoch 3/50
804/804 [==============================] - 3s 4ms/step - loss: 0.4496 - accuracy: 0.7792
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4477 - accuracy: 0.7818
Epoch 5/50
798/804 [============================>.] - ETA: 0s - loss: 0.4460 - accuracy: 0.7818
Epoch 5: loss improved from inf to 0.44575, saving model to /content/drive/MyDrive/weights.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.4458 - accuracy: 0.7820
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4451 - accuracy: 0.7832
Epoch 7/50
804/804 [==============================] - 3s 3ms/step - loss: 0.4442 - accuracy: 0.7838
Epoch 8/50
804/804 [==============================] - 3s 3ms/step - loss: 0.4441 - accuracy: 0.7843
Epoch 9/50
804/804 

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {round(model_loss,3)}, Accuracy: {round(model_accuracy,3)}")

268/268 - 1s - loss: 0.4600 - accuracy: 0.7789 - 544ms/epoch - 2ms/step
Loss: 0.46, Accuracy: 0.779


In [ ]:
# Create a DataFrame containing training history
import matplotlib.pyplot as plt
history_df = pd.DataFrame(fit_model.history, index=range(1,len(fit_model.history["loss"])+1))

# Plot the accuracy
history_df.plot(y="accuracy")
plt.show()

In [ ]:
# Export our model to HDF5 file
nn.save('/content/drive/MyDrive/AlphabetSoupCharity.h5')